In [4]:
import pandas as pd
import numpy as np
import zipfile,os
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.preprocessing import LabelEncoder
!pip install surprise
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
import re
import random

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
phone = '/content/drive/MyDrive/archive_7.zip'
phoneRef = zipfile.ZipFile(phone, 'r')
phoneRef.extractall('/content')
phoneRef.close()

In [36]:
dataframe = pd.read_csv('mobiles1.csv')
dataframe

,battery,camera,display,memory,name,price,processor,rating,reviews,warranty
0,5000 mAh Battery,12MP + 2MP | 8MP Front Camera,15.8 cm (6.22 inch) HD+ Display,4 GB RAM | 64 GB ROM | Expandable Upto 512 GB,"Redmi 8 (Ruby Red, 64 GB)",9999,Qualcomm Snapdragon 439 Processor,4.4,"55,078 Reviews",Brand Warranty of 1 Year Available for Mobile ...
1,5000 mAh Battery,12MP + 8MP + 2MP + 2MP | 8MP Front Camera,16.56 cm (6.52 inch) HD+ Display,4 GB RAM | 64 GB ROM,"Realme 5i (Aqua Blue, 64 GB)",10999,Qualcomm Snapdragon 665 2 GHz Processor,4.5,"20,062 Reviews",Sunrise Design
2,5000 mAh Battery,12MP + 8MP + 2MP + 2MP | 8MP Front Camera,16.56 cm (6.52 inch) HD+ Display,4 GB RAM | 128 GB ROM,"Realme 5i (Aqua Blue, 128 GB)",11999,Qualcomm Snapdragon 665 (2 GHz) Processor,4.5,"20,062 Reviews",Sunrise Design
3,5000 mAh Battery,12MP + 8MP + 2MP + 2MP | 8MP Front Camera,16.56 cm (6.52 inch) HD+ Display,4 GB RAM | 128 GB ROM,"Realme 5i (Forest Green, 128 GB)",11999,Qualcomm Snapdragon 665 (2 GHz) Processor,4.5,"20,062 Reviews",Sunrise Design
4,4000 mAh Battery,13MP + 2MP | 5MP Front Camera,15.49 cm (6.1 inch) HD+ Display,3 GB RAM | 32 GB ROM | Expandable Upto 256 GB,"Realme C2 (Diamond Blue, 32 GB)",7499,MediaTek P22 Octa Core 2.0 GHz Processor,4.4,"10,091 Reviews",Dual Nano SIM slots and Memory Card Slot
...,...,...,...,...,...,...,...,...,...,...
979,2000 mAh Battery,5MP Rear Camera | 2MP Front Camera,12.7 cm (5 inch) FWVGA Display,1 GB RAM | 8 GB ROM | Expandable Upto 32 MB,"Micromax Bharat 4 (Black, 8 GB)",3590,12 Months Brand Warranty,3.8,105 Reviews,NaN
980,2680 mAh Li-Ion Battery,13MP Rear Camera | 5MP Front Camera,13.21 cm (5.2 inch) Full HD Display,3 GB RAM | 32 GB ROM,"Nextbit Robin (Ember, 32 GB)",19999,Qualcomm Snapdragon 808 MSM8992 Processor,4.0,516 Reviews,Brand Warranty of 1 Year
981,4550 mAh Battery,13MP + 5MP | 20MP Front Camera,15.24 cm (6 inch) Full HD Display,4 GB RAM | 64 GB ROM | Expandable Upto 256 GB,"Gionee A1 Plus (Mocha Gold, 64 GB)",10499,Helio P25 MT 6757CD Processor,4.1,710 Reviews,Brand Warranty of 1 Year Available for Mobile ...
982,2100 mAh Li-Ion Battery,8MP Rear Camera | 2MP Front Camera,12.7 cm (5 inch) HD Display,1 GB RAM | 8 GB ROM | Expandable Upto 32 GB,"XOLO Omega 5.0 (Black, 8 GB)",8990,MTK 6592M Processor,3.8,81 Reviews,1 Year Manufacturer Warranty


In [37]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984 entries, 0 to 983
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   battery    984 non-null    object 
 1   camera     984 non-null    object 
 2   display    984 non-null    object 
 3   memory     984 non-null    object 
 4   name       984 non-null    object 
 5   price      984 non-null    int64  
 6   processor  983 non-null    object 
 7   rating     971 non-null    float64
 8   reviews    971 non-null    object 
 9   warranty   836 non-null    object 
dtypes: float64(1), int64(1), object(8)
memory usage: 77.0+ KB


In [38]:
dataframe = dataframe.drop(columns=['warranty'])

In [39]:
print('Jumlah Smartphone:', len(dataframe.name.unique()))

Jumlah Smartphone: 878


In [40]:
split_memory = dataframe['memory'].str.split('|', 1, expand=True)

dataframe['RAM(GB)'] = split_memory[0]
dataframe['ROM(GB)'] = split_memory[1]
dataframe.head()

,battery,camera,display,memory,name,price,processor,rating,reviews,RAM(GB),ROM(GB)
0,5000 mAh Battery,12MP + 2MP | 8MP Front Camera,15.8 cm (6.22 inch) HD+ Display,4 GB RAM | 64 GB ROM | Expandable Upto 512 GB,"Redmi 8 (Ruby Red, 64 GB)",9999,Qualcomm Snapdragon 439 Processor,4.4,"55,078 Reviews",4 GB RAM,64 GB ROM | Expandable Upto 512 GB
1,5000 mAh Battery,12MP + 8MP + 2MP + 2MP | 8MP Front Camera,16.56 cm (6.52 inch) HD+ Display,4 GB RAM | 64 GB ROM,"Realme 5i (Aqua Blue, 64 GB)",10999,Qualcomm Snapdragon 665 2 GHz Processor,4.5,"20,062 Reviews",4 GB RAM,64 GB ROM
2,5000 mAh Battery,12MP + 8MP + 2MP + 2MP | 8MP Front Camera,16.56 cm (6.52 inch) HD+ Display,4 GB RAM | 128 GB ROM,"Realme 5i (Aqua Blue, 128 GB)",11999,Qualcomm Snapdragon 665 (2 GHz) Processor,4.5,"20,062 Reviews",4 GB RAM,128 GB ROM
3,5000 mAh Battery,12MP + 8MP + 2MP + 2MP | 8MP Front Camera,16.56 cm (6.52 inch) HD+ Display,4 GB RAM | 128 GB ROM,"Realme 5i (Forest Green, 128 GB)",11999,Qualcomm Snapdragon 665 (2 GHz) Processor,4.5,"20,062 Reviews",4 GB RAM,128 GB ROM
4,4000 mAh Battery,13MP + 2MP | 5MP Front Camera,15.49 cm (6.1 inch) HD+ Display,3 GB RAM | 32 GB ROM | Expandable Upto 256 GB,"Realme C2 (Diamond Blue, 32 GB)",7499,MediaTek P22 Octa Core 2.0 GHz Processor,4.4,"10,091 Reviews",3 GB RAM,32 GB ROM | Expandable Upto 256 GB


In [41]:
data_battery= dataframe['battery'].str.split('(\d+)',  expand=True)
data_ram= dataframe['RAM(GB)'].str.split('(\d+)',  expand=True)
data_rom= dataframe['ROM(GB)'].str.split('(\d+)',  expand=True)

dataframe['battery(mAh)'] = pd.to_numeric(data_battery[1], downcast='float')
dataframe['RAM(GB)'] = pd.to_numeric(data_ram[1], downcast='integer')
dataframe['ROM(GB)'] = pd.to_numeric(data_rom[1], downcast='integer')
dataframe['price(IDR)'] = (dataframe['price'] * 191.31 )

In [42]:
dataframe = dataframe.drop(columns=['battery',
                                    'price',
                                    'memory'])
dataframe

,camera,display,name,processor,rating,reviews,RAM(GB),ROM(GB),battery(mAh),price(IDR)
0,12MP + 2MP | 8MP Front Camera,15.8 cm (6.22 inch) HD+ Display,"Redmi 8 (Ruby Red, 64 GB)",Qualcomm Snapdragon 439 Processor,4.4,"55,078 Reviews",4,64,5000.0,1912908.69
1,12MP + 8MP + 2MP + 2MP | 8MP Front Camera,16.56 cm (6.52 inch) HD+ Display,"Realme 5i (Aqua Blue, 64 GB)",Qualcomm Snapdragon 665 2 GHz Processor,4.5,"20,062 Reviews",4,64,5000.0,2104218.69
2,12MP + 8MP + 2MP + 2MP | 8MP Front Camera,16.56 cm (6.52 inch) HD+ Display,"Realme 5i (Aqua Blue, 128 GB)",Qualcomm Snapdragon 665 (2 GHz) Processor,4.5,"20,062 Reviews",4,128,5000.0,2295528.69
3,12MP + 8MP + 2MP + 2MP | 8MP Front Camera,16.56 cm (6.52 inch) HD+ Display,"Realme 5i (Forest Green, 128 GB)",Qualcomm Snapdragon 665 (2 GHz) Processor,4.5,"20,062 Reviews",4,128,5000.0,2295528.69
4,13MP + 2MP | 5MP Front Camera,15.49 cm (6.1 inch) HD+ Display,"Realme C2 (Diamond Blue, 32 GB)",MediaTek P22 Octa Core 2.0 GHz Processor,4.4,"10,091 Reviews",3,32,4000.0,1434633.69
...,...,...,...,...,...,...,...,...,...,...
979,5MP Rear Camera | 2MP Front Camera,12.7 cm (5 inch) FWVGA Display,"Micromax Bharat 4 (Black, 8 GB)",12 Months Brand Warranty,3.8,105 Reviews,1,8,2000.0,686802.90
980,13MP Rear Camera | 5MP Front Camera,13.21 cm (5.2 inch) Full HD Display,"Nextbit Robin (Ember, 32 GB)",Qualcomm Snapdragon 808 MSM8992 Processor,4.0,516 Reviews,3,32,2680.0,3826008.69
981,13MP + 5MP | 20MP Front Camera,15.24 cm (6 inch) Full HD Display,"Gionee A1 Plus (Mocha Gold, 64 GB)",Helio P25 MT 6757CD Processor,4.1,710 Reviews,4,64,4550.0,2008563.69
982,8MP Rear Camera | 2MP Front Camera,12.7 cm (5 inch) HD Display,"XOLO Omega 5.0 (Black, 8 GB)",MTK 6592M Processor,3.8,81 Reviews,1,8,2100.0,1719876.90


In [43]:
loc_data=dataframe.loc[
                       (dataframe['battery(mAh)'] >= 3000) &
                       (dataframe['RAM(GB)'] >= 3) &
                       (dataframe['ROM(GB)'] >= 32)
                       ]

In [44]:
newDataframe = loc_data
newDataframe.head()

,camera,display,name,processor,rating,reviews,RAM(GB),ROM(GB),battery(mAh),price(IDR)
0,12MP + 2MP | 8MP Front Camera,15.8 cm (6.22 inch) HD+ Display,"Redmi 8 (Ruby Red, 64 GB)",Qualcomm Snapdragon 439 Processor,4.4,"55,078 Reviews",4,64,5000.0,1912908.69
1,12MP + 8MP + 2MP + 2MP | 8MP Front Camera,16.56 cm (6.52 inch) HD+ Display,"Realme 5i (Aqua Blue, 64 GB)",Qualcomm Snapdragon 665 2 GHz Processor,4.5,"20,062 Reviews",4,64,5000.0,2104218.69
2,12MP + 8MP + 2MP + 2MP | 8MP Front Camera,16.56 cm (6.52 inch) HD+ Display,"Realme 5i (Aqua Blue, 128 GB)",Qualcomm Snapdragon 665 (2 GHz) Processor,4.5,"20,062 Reviews",4,128,5000.0,2295528.69
3,12MP + 8MP + 2MP + 2MP | 8MP Front Camera,16.56 cm (6.52 inch) HD+ Display,"Realme 5i (Forest Green, 128 GB)",Qualcomm Snapdragon 665 (2 GHz) Processor,4.5,"20,062 Reviews",4,128,5000.0,2295528.69
4,13MP + 2MP | 5MP Front Camera,15.49 cm (6.1 inch) HD+ Display,"Realme C2 (Diamond Blue, 32 GB)",MediaTek P22 Octa Core 2.0 GHz Processor,4.4,"10,091 Reviews",3,32,4000.0,1434633.69


In [45]:
newDataframe.isnull().sum()

camera           0
display          0
name             0
processor        1
rating          13
reviews         13
RAM(GB)          0
ROM(GB)          0
battery(mAh)     0
price(IDR)       0
dtype: int64

In [46]:
newDataframe.loc[:,'processor'] = newDataframe['processor'].fillna('Unknown')
newDataframe.loc[:,'rating'] = newDataframe['rating'].fillna('Unknown')
newDataframe.loc[:,'reviews'] = newDataframe['reviews'].fillna('Unknown')
newDataframe.isnull().sum()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


camera          0
display         0
name            0
processor       0
rating          0
reviews         0
RAM(GB)         0
ROM(GB)         0
battery(mAh)    0
price(IDR)      0
dtype: int64

In [47]:
newDataframe = newDataframe.reset_index()
newDataframe = newDataframe.rename(columns={"index":"id"})
newDataframe['id'] = newDataframe.index + 0

In [48]:
newDataframe

,id,camera,display,name,processor,rating,reviews,RAM(GB),ROM(GB),battery(mAh),price(IDR)
0,0,12MP + 2MP | 8MP Front Camera,15.8 cm (6.22 inch) HD+ Display,"Redmi 8 (Ruby Red, 64 GB)",Qualcomm Snapdragon 439 Processor,4.4,"55,078 Reviews",4,64,5000.0,1912908.69
1,1,12MP + 8MP + 2MP + 2MP | 8MP Front Camera,16.56 cm (6.52 inch) HD+ Display,"Realme 5i (Aqua Blue, 64 GB)",Qualcomm Snapdragon 665 2 GHz Processor,4.5,"20,062 Reviews",4,64,5000.0,2104218.69
2,2,12MP + 8MP + 2MP + 2MP | 8MP Front Camera,16.56 cm (6.52 inch) HD+ Display,"Realme 5i (Aqua Blue, 128 GB)",Qualcomm Snapdragon 665 (2 GHz) Processor,4.5,"20,062 Reviews",4,128,5000.0,2295528.69
3,3,12MP + 8MP + 2MP + 2MP | 8MP Front Camera,16.56 cm (6.52 inch) HD+ Display,"Realme 5i (Forest Green, 128 GB)",Qualcomm Snapdragon 665 (2 GHz) Processor,4.5,"20,062 Reviews",4,128,5000.0,2295528.69
4,4,13MP + 2MP | 5MP Front Camera,15.49 cm (6.1 inch) HD+ Display,"Realme C2 (Diamond Blue, 32 GB)",MediaTek P22 Octa Core 2.0 GHz Processor,4.4,"10,091 Reviews",3,32,4000.0,1434633.69
...,...,...,...,...,...,...,...,...,...,...,...
726,726,48MP + 5MP | 16MP Front Camera,16.51 cm (6.5 inch) Full HD+ Display,"OPPO F11 Pro (Thunder Black, 64 GB)",MediaTek Helio P70 Octa Core 2.1 GHz Processor,4.4,"11,506 Reviews",6,64,4000.0,4015596.90
727,727,48MP + 5MP | 16MP Front Camera,16.51 cm (6.5 inch) Full HD+ Display,"OPPO F11 Pro (Aurora Green, 64 GB)",MediaTek Helio P70 Octa Core 2.1 GHz Processor,4.4,"11,506 Reviews",6,64,4000.0,4780836.90
728,728,12MP + 5MP | 13MP Front Camera,15.9 cm (6.26 inch) Full HD+ Display,"Asus ZenFone Max Pro M2 (Blue, 64 GB)",Qualcomm Snapdragon 660 Octa Core Processor,4.3,"1,955 Reviews",6,64,5000.0,2678148.69
729,729,13MP + 5MP | 20MP Front Camera,15.24 cm (6 inch) Full HD Display,"Gionee A1 Plus (Mocha Gold, 64 GB)",Helio P25 MT 6757CD Processor,4.1,710 Reviews,4,64,4550.0,2008563.69


In [29]:
rating = newDataframe['rating']
phoneName = newDataframe['name']

In [53]:
phoneRating = book_rating.groupby('ISBN')['Book-Rating'].mean().sort_values(ascending=False)
phoneRating

SyntaxError: ignored